# Import Modules

In [ ]:
import sys
sys.path.append('../')

import Levels
import Results
from RMatrix import PTBayes
import Resonances
from SpinGroups import SpinGroups

In [ ]:
%matplotlib widget

# Basic Fitting Example (WigBayes)

### Setting Mean Parameter Data

In [ ]:
A  = 181
EB = (1e-5,2550)
pM    = [0.12547, 0.14404]
FreqF = 1/15
Freq  = [1/8.9868, 1/8.3065]
Gnm   = [44.11355, 33.38697]
Ggm   = [55.00000, 55.00000]
dfn   = [1, 1]
dfg   = [250, 250]
l     = [0, 0]
j     = [3.0, 4.0]

SGs = SpinGroups.make(l, j)
MP = Resonances.MeanParameters(Freq=Freq, Gnm=Gnm, nDOF=dfn, Ggm=Ggm, gDOF=dfg, A=A, sg=SGs, EB=EB, FreqF=FreqF)

### Sample Resonance Ladder

In [ ]:
Res, Types, Missed_Res, Missed_Types = MP.sample()

In [ ]:
Prior, TPPrior = PTBayes(Res, MP)
# runMaster = Levels.RunMaster(Res.E, MP.EB, Prior, TPPrior, MP.FreqAll)
distributions = MP.distributions('Wigner')
runMaster = Levels.RunMaster(Res.E, MP.EB, Prior, TPPrior, distributions, MP.FreqF)
Posterior, log_total_probability = runMaster.WigBayes(return_log_tot_prob=True, verbose=True)

### Results:

In [ ]:
Results.PrintScore(Prior, Types, 'PT-only')
Results.PrintScore(Posterior, Types, 'Wigner+PT')
print()
Results.ConfusionMatrix(Posterior, Types, ['A', 'B', 'F'])
print()
Results.ProbCorrPlot(Posterior, Types, ['A', 'B', 'F'])

In [ ]:
print(Posterior)

# Basic Spingroup Sampling (WigSample)

...